In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline

folder=tb.latest_data('tail')

print folder
name = 'adwindata'
TH_or_HH = 'TH'
ZPL = True
a = pqsequence.TailAnalysis(folder, hdf5_mode='r')#, pq_folder='bs_remote')


a.get_sweep_pts()
a.get_sweep_idxs(noof_syncs_per_sweep_pt=250)
a.get_readout_results(name)

if a.g.attrs['general_sweep_name']=='eom_off_amplitude':
    print 'sweeping EOM off voltage'
    a.get_tail_vs_sweep(0,1680,20, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=True)
else:
    if TH_or_HH == 'TH':
        if ZPL:
            a.get_tail_vs_sweep(1,1690,50, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=True)
        else:
            a.get_tail_vs_sweep(0,1600,200,pq_binsize_ns=1,hist_binsize_ns=1.,verbose = True)
    else:
        print 'HH'
        a.get_tail_vs_sweep(0,2783e3,40e3, pq_binsize_ns=1, hist_binsize_ns=200.,verbose=True)



save=True
a.plot_tail_cts_vs_sweep(save=save, channels=1)
a.plot_tail_hists_vs_sweep(save=save)
a.plot_tail_hist_integated(save=save, log_plot=True)
a.plot_tail_hist_all(save=save, log_plot=1, offset=0, indices=[np.argmax(a.tail_cts_per_sweep_idx)])

In [ ]:
## ZPL Counts in tail?
a.get_tail_vs_sweep(1,3076,100, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=True)
a.plot_tail_cts_vs_sweep(save=False, channels=1)
a.plot_tail_hists_vs_sweep(save=False)
a.plot_tail_hist_integated(save=False, log_plot=True)
a.plot_tail_hist_all(save=False, log_plot=1, offset=0, indices=[np.argmax(a.tail_cts_per_sweep_idx)])

In [ ]:
from analysis.lib.fitting import fit, common, rabi
ax=a.plot_tail_cts_vs_sweep(save=False, ret='ax')
x=a.g.attrs['general_sweep_pts']
y=a.tail_cts_per_sweep_idx
ff = common.fit_parabole
#o + A * (x-c)**2
#['g_o', 'g_A', 'g_c']
args = [0,np.max(y), x[np.argmin(y)]]
print args
fitres = fit.fit1d(x, y, ff, *args, fixed = [],
                   do_print = True, ret = True)
plot_pts=100
x_p=np.linspace(min(x),max(x),plot_pts)
if fitres['success']:
    f_p = fitres['fitfunc'](x_p)*1e4
    ax.plot(x_p,f_p)

In [ ]:
from analysis.lib.fitting import fit, common, rabi
from analysis.lib.tools import plot
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
fs=zeros(a.sweep_length)
taus = zeros(a.sweep_length)
for i in range(a.sweep_length):
    x=a.tail_hist_b[:-1]        
    y=a.tail_hist_h[i]
    fit_result = fit.fit1d(x, y, rabi.fit_rabi_damped_exp_with_offset, 
            1/5.,1000,0,10., x[0], ret = True, color = 'r', linewidth = 2.0)
    fs[i] = fit_result['params_dict']['f']
    taus[i] = fit_result['params_dict']['tau']
    plot.plot_fit1d(fit_result, np.linspace(x[0],x[-1],201), ax=ax,
        plot_data=True, print_info=False)
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
#print taus, fs
ax.plot(a.sweep_pts, fs)
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
ax.plot(a.sweep_pts, taus)


In [ ]:
a.get_cr_results(name)
a.plot_cr_vs_sweep()

In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
#folder=tb.latest_data('Bell')
folder=tb.data_from_time('20141012023031')
#folder=r'D:\measuring\data\20140519\192030_Bell_lt3_tailS_The111no2_SIL1_Ey_+11deg'
name = 'ssro'
a = pqsequence.TailAnalysis(folder, hdf5_mode='r', pq_folder='bs_remote')
a.reps=200000
a.plot_histogram(0, start=5.350e6, length=1000000, hist_binsize=1000, save=False)

In [ ]:
from analysis.lib.pq import pq_tools
reload(pq_tools)

In [ ]:
fltr0=pq_tools.filter_synctimes(a.pqf, 5371.5e3, 5397e3,15,600e3)
fltr1=pq_tools.filter_synctimes(a.pqf, 5374e3, 5399.5e3,15,600e3)
cs=pq_tools.get_coincidences(a.pqf, fltr0, fltr1)
dts_ns = cs[:,0]*1e-3
len(cs)

In [ ]:
binsize=10
G2_BINEDGES = arange(-19000,19000,binsize)
#G2_BINEDGES = np.hstack((arange(-700,-500,binsize),arange(-100,100,binsize),arange(500,700,binsize)))
#print G2_BINEDGES
figure(figsize=(20,5))
b_=hist(dts_ns, bins=G2_BINEDGES)

In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
folder=tb.latest_data('Bell')
folder=tb.data_from_time('20140710153945')
a = sequence.SequenceAnalysis(folder)
a.get_cr_results(name)
a.get_sweep_pts()
a.plot_cr_vs_sweep()
a.finish()

In [ ]:
from analysis.lib.fitting import fit, common
from analysis.lib.tools import plot
fig, ax = plt.subplots(1,1, figsize=(4.5,4))
fs=np.zeros(a.sweep_length)
taus = np.zeros(a.sweep_length)
for i in range(a.sweep_length):
    x=a.tail_hist_b[:-1]        
    y=a.tail_hist_h[i]
    p0, fitfunc, fitfunc_str = common.fit_general_exponential(0.,300,5427.,2., 2.)
#     ax.plot(np.linspace(x[0],x[-1],201), fitfunc(np.linspace(x[0],x[-1],201)), ':', lw=2)
    fit_result = fit.fit1d(x, y, None, p0=p0, fitfunc=fitfunc, 
            ret = True, color = 'r', linewidth = 2.0,fixed=[4],do_print=True)
    
    plot.plot_fit1d(fit_result, np.linspace(x[0],x[-1],201), ax=ax,
        plot_data=True, print_info=True)
#     fs[i] = fit_result['params_dict']['f']
#     taus[i] = fit_result['params_dict']['tau']
# fig, ax = plt.subplots(1,1, figsize=(4.5,4))
# #print taus, fs
# ax.plot(a.sweep_pts, fs)
# fig, ax = plt.subplots(1,1, figsize=(4.5,4))
# ax.plot(a.sweep_pts, taus)

In [ ]:
624-166

In [ ]:
listy = [0,1,2,3]
listy.insert(-1,5)
print listy